In [1]:
import h5py
import re
import hickle as hkl
import torch
import torch.nn as nn
from torch.legacy.nn import Reshape
import graphviz
import torch.nn.functional as F
from torch.autograd import Variable
#from visualize import make_dot
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as utils
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import imresize, imread, imshow
import time
import logging
from math import log,sqrt

In [2]:
torch.cuda.current_device()

0

In [2]:
dense161 = models.densenet161(pretrained=True)
#dense161

In [ ]:
dense161


In [3]:
def InitializeWeights(mod):
    for m in mod.modules():
        if isinstance(m,nn.Conv2d):
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            #print m.weight.size(), m.out_channels, m.in_channels
            m.weight.data.normal_(0,sqrt(2./n))
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
    return mod

In [22]:
class MyModel4(nn.Module):
    def __init__(self, pretrained_model):
        super(MyModel4, self).__init__()
        self.pretrained_model = nn.Sequential(*list(dense161.children())[:-1])

        #self.unpool1 = nn.UpsamplingBilinear2d(scale_factor=2)
        #self.unpool2 = nn.UpsamplingBilinear2d(size=(60,80))
        #self.conv0 = nn.Conv2d(2208,1024,1)
        ##self.conv1 = nn.Conv2d(1024,512,1)
        self.conv1 = InitializeWeights(nn.Conv2d(2208,1024,1))
        self.conv2 = InitializeWeights(nn.Conv2d(1024,128,5))
        self.conv3 = InitializeWeights(nn.Conv2d(128,16,1))
        self.fc1 = nn.Linear(96, 2)
       # self.conv2 = nn.Conv2d(512,256,5,padding=(2,2))
        #self.conv3 = nn.Conv2d(256,128,5,padding=(4,2))
        #self.conv4 = nn.Conv2d(128,1,3,padding=(1,1))
       # #self.conv5 = nn.Conv2d(128,1,3,padding=(1,1))
        self.bnorm1 = InitializeWeights(nn.BatchNorm2d(1024))
        self.bnorm2 = InitializeWeights(nn.BatchNorm2d(128))
        self.bnorm3 = InitializeWeights(nn.BatchNorm2d(16))
        #self.relu = nn.ReLU()
        
    def unpool_conv(self,inp1,conv,unpool):
        #x = inp1
        x = unpool(inp1)
        x = F.relu(conv(x))
        return x
    
    #below for testing purpose only...
    def unpool_conv1(self,inp1,conv,unpool):
        x = inp1
        #x = unpool(inp1)
        x = F.relu(conv(x))
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

        
    def forward(self, x):
        x = F.relu(self.pretrained_model(x))
        x=F.relu(self.bnorm1(self.conv1(x)))
        x=F.relu(self.bnorm2(self.conv2(x)))
        x=F.relu(self.bnorm3(self.conv3(x)))
        x = x.view(-1, self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        
        #x = self.conv0(x)
        #x = self.bnorm(x)
       # x = self.unpool_conv(x,self.conv1,self.unpool1)
       # x = self.unpool_conv(x,self.conv2,self.unpool1)
        #x = self.unpool_conv(x,self.conv3,self.unpool1)
       # #x = self.unpool_conv(x,self.conv4,self.unpool2)
       # #x = self.unpool2(x)
        #x = self.conv4(x)
        #x = x.view(-1,60,80)
        
        return x
net = MyModel4(dense161)
#print(net)

In [ ]:
#model4 = MyModel4(dense161)
#model4.forward(Variable(torch.randn(2,3,240,320))).size()

In [23]:
input=Variable(torch.randn(1,3,180,320))
output=net(input)
print(output.size())

torch.Size([1, 2])


In [24]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

496
torch.Size([96, 3, 7, 7])


In [7]:
sum1 = 0
        
print("Number of layers ---> ",len(list(net.parameters())))
for params in net.parameters():
    if params.requires_grad == True:
        sum1 += params.numel()
    
print("Total number of parameters ---> ",sum1)

Number of layers --->  496
Total number of parameters --->  32015732


In [25]:
input = Variable(torch.randn(5, 3, 180, 320))
out = net(input)
print(out)

Variable containing:
 0.0000  0.3506
 0.0667  0.0000
 0.0000  0.0000
 0.0000  0.0672
 0.0000  0.0000
[torch.FloatTensor of size 5x2]



In [9]:
file = h5py.File('./DATASET/CODE/NewTrainData.h5')
xtrainT = torch.from_numpy(np.array(file['xtrain'],dtype=np.float32)).float()
ytrainT = torch.from_numpy(np.array(file['ytrain'],dtype=np.float32)).float()
#xtrain = np.array(file['xtrain'],dtype=np.float32)
#ytrain = np.array(file['ytrain'],dtype=np.float32)

In [10]:
file = h5py.File('./DATASET/CODE/NewTestData.h5')
xtestT = torch.from_numpy(np.array(file['xtest'],dtype=np.float32)).float()
ytestT = torch.from_numpy(np.array(file['ytest'],dtype=np.float32)).float()
#xtest = np.array(file['xtest'],dtype=np.float32)
#ytest = np.array(file['ytest'],dtype=np.float32)

In [11]:
print(xtrainT.size(), ytrainT.size(), xtestT.size(), ytestT.size())

torch.Size([20100, 3, 180, 320]) torch.Size([20100, 2, 3]) torch.Size([1560, 3, 180, 320]) torch.Size([1560, 2, 3])


In [12]:
def batch_rgb_to_bgr(batch):
    #print(batch.size())
    (r, g, b) = torch.chunk(batch, 3, 1)
    #print(r.size())
    batch1 = torch.cat((b, g, r),1)
    #print(batch1.size())
    return batch1

In [13]:
xtrainT = batch_rgb_to_bgr(xtrainT)
xtestT = batch_rgb_to_bgr(xtestT)
print(xtrainT.size(), xtestT.size())

torch.Size([20100, 3, 180, 320]) torch.Size([1560, 3, 180, 320])


In [14]:
xtrainT = torch.div(xtrainT,255.0)
xtestT = torch.div(xtestT,255.0)

In [15]:
print(torch.min(xtrainT), torch.max(xtrainT), torch.min(xtestT), torch.max(xtestT))

0.0 1.0 0.0 1.0


In [16]:
print(xtrainT.size(), ytrainT.size(), xtestT.size(), ytestT.size())

torch.Size([20100, 3, 180, 320]) torch.Size([20100, 2, 3]) torch.Size([1560, 3, 180, 320]) torch.Size([1560, 2, 3])


In [17]:
class Normalize(object):
    """
    Normalize an tensor image with mean and standard deviation.
    Given mean: (R, G, B) and std: (R, G, B),
    will normalize each channel of the torch.*Tensor, i.e.
    channel = (channel - mean) / std
    Args:
        mean (sequence): Sequence of means for R, G, B channels respecitvely.
        std (sequence): Sequence of standard deviations for R, G, B channels
            respecitvely.
    """

    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        # TODO: make efficient
        for t, m, s in zip(tensor, self.mean, self.std):
            t.sub_(m).div_(s)
        return tensor

In [18]:
mn = [0.406,0.456,0.485]
sd = [0.225,0.224,0.229]
norm = Normalize(mn,sd)
xtrainT = norm(xtrainT)
xtestT = norm(xtestT)

In [19]:
print(torch.min(xtrainT), torch.max(xtrainT), torch.min(xtestT), torch.max(xtestT))

-2.1179039478302 2.640000104904175 -2.1007792949676514 2.640000104904175


In [20]:
##def train(model, loss, optimizer, x_val, y_val, validPixel, batch_sz):
def train(model, loss, optimizer, x_val, y_val):
    x = Variable(x_val,requires_grad = False).cuda()
    y = Variable(y_val,requires_grad = False).cuda()
    
    optimizer.zero_grad()
    
    fx = model.forward(x)
    
    #print fx.data[0][0][64][87]
    #fx = model5.forward(Variable(xtest2[start:end], volatile=True).cuda())
    ##output = loss.forward(fx,y,validPixel,batch_sz)
    output = loss.forward(fx,y)
    #output = loss(fx, y)
    output.backward()
    
    optimizer.step()
    
    return output.data[0]

In [28]:
#custom loss function.... this will be reverse Huber...

class VectorLoss(nn.Module):
    def __init__(self):
        super(VectorLoss, self).__init__()
        
    def forward(self,inp, tar):
        N = ((tar[:,2]-tar[:,0])**2+(tar[:,3]-tar[:,1])**2)**0.5
        Px = (tar[:,2]-tar[:,0])/N
        Py = (tar[:,3]-tar[:,1])/N        
        loss = (Px-inp[:,0])**2+(Py-inp[:,1])**2
        return torch.mean(loss)

In [22]:
#custom loss function.... this will be reverse Huber...

class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
        
    def forward(self,inp, tar):
        #target is the ground truth value...
        #k = torch.mean(inp[:,0])
        '''
        if (k >= 1.48 and k <= 1.65):
            diff = torch.abs(tar[:,1]-inp[:,1])
            loss = torch.mean(torch.pow(diff,2))
        else:
        '''
        diff = torch.abs(tar[:,0]-inp[:,0]) #*(180/np.pi)
        loss = torch.mean(torch.pow(diff,2))
        #print(loss)
        return loss
        '''
        c1 = c.data[0] 
        temp = diff > c1
        check1 = torch.prod(temp)
        
        if check1 == 0:
            lossval = torch.mean(diff)
        else:
            temp4 = torch.pow(diff,2)
            d = torch.pow(c,2)
            temp4 = temp4.add(d.expand_as(temp4))
            lossval = torch.mean(temp4/(2*c))
        return lossval
        '''

In [20]:
#custom loss function.... this will be reverse Huber...

class EuclideanLoss(nn.Module):
    def __init__(self):
        super(EuclideanLoss, self).__init__()
        
    def forward(self,inp, tar):
        #target is the ground truth value...
        #k = torch.mean(inp[:,0])
        '''
        if (k >= 1.48 and k <= 1.65):
            diff = torch.abs(tar[:,1]-inp[:,1])
            loss = torch.mean(torch.pow(diff,2))
        else:
        '''
        #diff = torch.abs(tar[:,0]-inp[:,0]) #*(180/np.pi)
        #loss = torch.mean(torch.pow(diff,2))
        
        
        
        #print(tar.size(),inp.size())
        diff1 = ((tar[:,0]-inp[:,0])/320)**2+((tar[:,1]-inp[:,1])/180)**2
        diff2 = ((tar[:,2]-inp[:,2])/320)**2+((tar[:,3]-inp[:,3])/180)**2
        diff=diff1+diff2
        loss = torch.mean(diff)
        
        
        
        #print(loss)
        return loss
        '''
        c1 = c.data[0] 
        temp = diff > c1
        check1 = torch.prod(temp)
        
        if check1 == 0:
            lossval = torch.mean(diff)
        else:
            temp4 = torch.pow(diff,2)
            d = torch.pow(c,2)
            temp4 = temp4.add(d.expand_as(temp4))
            lossval = torch.mean(temp4/(2*c))
        return lossval
        '''

In [21]:
#alpha = torch.FloatTensor(ytrainT[5,0])
alpha = ytrainT[5,0]
#print(alpha.shape)
xt = torch.FloatTensor([np.cos(alpha),np.sin(alpha)])
print(ytrainT[5,0],xt.size())

1.5909278392791748 torch.Size([2])


In [22]:
alpha = ytrainT[5:10,0]
print(torch.cos(alpha[0:1]-alpha[1:2]))
xt = torch.stack([torch.cos(alpha[0:1]),torch.sin(alpha[0:1])])
xp = torch.stack([torch.cos(alpha[1:2]),torch.sin(alpha[1:2])])
print(xt[0],xt[1])
#print(los)


 0.9992
[torch.FloatTensor of size 1]


1.00000e-02 *
 -2.0130
[torch.FloatTensor of size 1]
 
 0.9998
[torch.FloatTensor of size 1]



In [23]:
class CosineLoss(nn.Module):
    def __init__(self):
        super(CosineLoss, self).__init__()
        
    def forward(self,inp, tar,batch_sz):
        alpha_t = tar[:,0]
        alpha_p = inp[:,0]
        #xt = torch.stack([torch.cos(alpha_t),torch.sin(alpha_t)])
        #xp = torch.stack([torch.cos(alpha_p),torch.sin(alpha_p)])
        #cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        #loss = cos(xt, xp)
        #return loss
        loss = Variable(torch.FloatTensor(batch_sz).zero_(), requires_grad=False).cuda()
        for i in range(batch_sz):          
            loss[i] = torch.cos(alpha_t[i:i+1]-alpha_p[i:i+1])
            
        lossval = 1.0-torch.mean(loss)    
        #print(lossval)
        return lossval
        

In [32]:
#MUST UNCOMMENT BELOW LINE...
    
net = net.cuda()

#loading the model after the weights of epoch50.. to check what loss the model gives if lr is taken as 0.0001
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

#criterion = RMSELoss()
#criterion = BerhuLoss()
#criterion = EuclideanLoss()
#criterion = nn.MSELoss()
criterion = VectorLoss()
#criterion = torch.nn.MSELoss(size_average=False)
#criterion = CustomLoss()
#criterion = CosineLoss()
criterion.cuda()

currepochloss = float('Inf')
#epochs, n_examples, i, batch_size, flag = 1,5900, 0, 5, 0
epochs, n_examples, i, batch_size, flag = 10, 20100, 0, 30, 0


while i != epochs:
    since = time.time()
    cost, batchloss = 0.0, 0.0
    num_batches = n_examples//batch_size
    #print num_batches    #indices = np.random.permutation(5600)
    #indices = np.random.permutation(3524)
    
    #indices = np.random.permutation(5900)
    indices = np.random.permutation(n_examples)
    samplesUnprocessed = np.size(indices)
    
    #batchwise training starts here...
    for k in range(num_batches):
        since1 = time.time()
       # print("bacth number:"+str(k))
        xtrain3 = torch.FloatTensor(batch_size,3,180,320)
        ytrain3 = torch.FloatTensor(batch_size,4)
        ##validPixel = torch.FloatTensor(batch_size,480,640)
        
        for ind in range(batch_size):
            #ind1 = np.random.randint(0,5599)
            ind1 = np.random.randint(0,samplesUnprocessed)
            #ind1 = np.random.randint(0,794)
            #ind1 = np.random.randint(0,794)            
            newxind = indices[ind1]            
            xtrain3[ind] = xtrainT[newxind]
            #ytrain3[ind] = ytrainT[newxind]
            ytrain3[ind,0] = ytrainT[newxind,0,0]
            ytrain3[ind,1] = ytrainT[newxind,0,1]
            ytrain3[ind,2] = ytrainT[newxind,1,0]
            ytrain3[ind,3] = ytrainT[newxind,1,1]        
            ##validPixel[ind] = imgValidTrain2[newxind]
            
            #print ytrain3[ind,0,0,0], ytrain2[newxind,0,0,0]
            indices = np.delete(indices,ind1)
            samplesUnprocessed = samplesUnprocessed - 1
        
        #start, end = k*batch_size, (k+1)*batch_size
        #batchloss = train(model5,criterion, optimizer, xtrain3, ytrain3, validPixel,batch_size)
        batchloss = train(net,criterion, optimizer, xtrain3, ytrain3)
        batch_time = time.time() - since1
        #cost += batchloss
        cost = (cost*k+batchloss)/(k+1)
        #print k,cost
        #print("No. of samples UnProcessed "+str(samplesUnprocessed))
        
    time_elapsed = time.time() - since
    epochloss = cost #/num_batches
    
    if epochloss < currepochloss:
        print('save the weights')
        torch.save(net.state_dict(),"./weights/VectorLoss/VectorLoss_20100_33.pth")
        flag = 0
        currepochloss = epochloss
    else:
        flag += 1
        
        if flag == 5:
            for p in optimizer.param_groups:
                lr2 = p['lr']
            newlr = lr2/5
            
            if newlr < 1e-15:
                print("Cant decrease further!!")
                newlr = 1e-15
            flag = 0 
            optimizer = optim.SGD(net.parameters(), lr=newlr, momentum=0.9)
            print("Learning rate changed from "+str(lr2)+" to "+str(newlr))
            
        print("Loss "+str(epochloss)+" is bigger than Loss "+str(currepochloss)+" in the prev epoch ")
        
    print('Loss = {:.8f} at epoch {:d} completed in {:.0f}m {:.0f}s'.format(epochloss,(i+1),(time_elapsed//60),(time_elapsed%60)))
    i += 1 

save the weights
Loss = 0.5753 at epoch 1 completed in 5m 37s
save the weights
Loss = 0.5753 at epoch 2 completed in 5m 36s
save the weights
Loss = 0.5753 at epoch 3 completed in 5m 38s
save the weights
Loss = 0.5753 at epoch 4 completed in 5m 35s
save the weights
Loss = 0.5753 at epoch 5 completed in 5m 35s
save the weights
Loss = 0.5753 at epoch 6 completed in 5m 37s
save the weights
Loss = 0.5753 at epoch 7 completed in 5m 37s
save the weights
Loss = 0.5753 at epoch 8 completed in 5m 36s
save the weights
Loss = 0.5753 at epoch 9 completed in 5m 37s
Loss 0.5752717153349914 is bigger than Loss 0.5752671838251513 in the prev epoch 
Loss = 0.5753 at epoch 10 completed in 5m 37s


In [33]:

for params in optimizer.param_groups:
    print(params['lr'])

0.001


In [ ]:
#net = net.cuda()
#net.load_state_dict(torch.load("./weights/corridor_new_data_bgr_600.pth"))

In [35]:
#testing of the architecture...
num_batches = 0
#6 evenly divides the test batch size..
test_batch_size = 10
n_examples = 1560
#finalpred = Variable(torch.zeros((n_examples,3,120,160)))
finalpred = Variable(torch.zeros((n_examples,2)))
print("finalpred size is ---> ", finalpred.size())

num_batches = n_examples//test_batch_size
print("num of batches --->", num_batches)
for k in range(num_batches):
    start, end = k*test_batch_size, (k+1)*test_batch_size
    output = net.forward(Variable(xtestT[start:end], volatile=True).cuda())
    finalpred[start:end] = output

finalpred size is --->  torch.Size([1560, 2])
num of batches ---> 156


In [36]:
data1 = finalpred.data.numpy()
print(data1.shape)

(1560, 2)


In [88]:
print(ytestT.size())
print(finalpred.size())
#dif1 = ((ytestT[0,0,0]-finalpred[0,0])/320)**2+((ytestT[0,1,0]-finalpred[0,1])/180)**2
#dif2 = ((ytestT[:,0,1]-finalpred.data[:,2])/320)**2+((ytestT[:,1,1]-finalpred.data[:,3])/180)**2
#print(dif1)
#print(dif2)

angle_pred=torch.atan((finalpred.data[:,3]-finalpred.data[:,1])/(finalpred.data[:,2]-finalpred.data[:,0]))
#print(ang*(180/np.pi))
#ang1=ang
for i in range(0,angle_pred.size(0)):
    if angle_pred[i]<0:
        angle_pred[i]+=np.pi
        
#print(angle_pred)        
angle_diff=torch.abs(ytestT[:,0,2]-angle_pred[:])
mse_angle=torch.mean(torch.pow(angle_diff,2)) 
abs_angle=torch.mean((angle_diff))
angle_diff*=180/np.pi
print (mse_angle*(180/np.pi))
print(abs_angle)

np.savetxt("pred_angle.csv", angle_diff.numpy(), delimiter=",")


torch.Size([1560, 2, 3])
torch.Size([1560, 4])
51.838359108884944
0.7197120178968478


In [47]:
final = finalpred.data
print(final.size())
Px1 = final[:,0]
Py1 = final[:,1]
N = Px1**2 + Py1**2
print(N)
Px1 = Px1/N
ang = torch.acos(Px1)
print(ang)
angle_diff=torch.abs(ytestT[:,0,2]-ang[:])
mse_angle=torch.mean(torch.pow(angle_diff,2)) 
abs_angle=torch.mean((angle_diff))
angle_diff*=180/np.pi
print (mse_angle*(180/np.pi))
print(abs_angle)

torch.Size([1560, 2])

 0.0050
 0.0079
 0.0000
   ⋮   
 0.0000
 0.6306
 0.0000
[torch.FloatTensor of size 1560]


    nan
    nan
    nan
   ⋮   
    nan
    nan
    nan
[torch.FloatTensor of size 1560]

nan
nan


In [89]:
#dif = torch.abs(finalpred.data[:,0]-ytestT[:,0])

    # diff1 = ((tar[:,0]-inp[:,0])/320)**2+((tar[:,1]-inp[:,1])/180)**2
     #diff2 = ((tar[:,2]-inp[:,2])/320)**2+((tar[:,3]-inp[:,3])/180)**2
      #  diff=diff1+diff2
       # loss = torch.mean(diff)

        
        
        
#dif1 = ((ytestT[:,0,0]-finalpred.data[:,0])/320)**2+((ytestT[:,1,0]-finalpred.data[:,1])/180)**2
#dif2 = ((ytestT[:,0,1]-finalpred.data[:,2])/320)**2+((ytestT[:,1,1]-finalpred.data[:,3])/180)**2
#dif=dif1+dif2
       
         
dif1 = ((ytestT[:,0,0]-finalpred.data[:,0]))**2+((ytestT[:,1,0]-finalpred.data[:,1]))**2
dif2 = ((ytestT[:,0,1]-finalpred.data[:,2]))**2+((ytestT[:,1,1]-finalpred.data[:,3]))**2
dif=dif1+dif2
        
        
        






print(dif.size())
np.savetxt("diff.csv", dif.numpy(), delimiter=",")
MSEloss = torch.mean(torch.pow(dif,2))
ABSloss = torch.mean(dif)
print("MSEloss=="+str(MSEloss),"ABSloss=="+str(ABSloss))

torch.Size([1560])
MSEloss==208950678.31407315 ABSloss==11412.042116854742


In [92]:
print(finalpred.data[0,0], finalpred.data[0,1], finalpred.data[0,2], finalpred.data[0,3])
print(ytestT[0,0,0],ytestT[0,1,0],ytestT[0,0,1],ytestT[0,1,1])

126.92657470703125 178.7459259033203 188.37425231933594 107.12451171875
157.1005859375 179.46665954589844 157.1005859375 79.46666717529297


In [31]:
n=887

print(finalpred.data[n,0]*(180/np.pi))
print(ytestT[n,0]*(180/np.pi))
print(ytestT[n,0]*(180/np.pi)-finalpred.data[n,0]*(180/np.pi))

84.39787451704454
43.8015182038607
-40.59635631318384


In [32]:
ABSloss*(180/np.pi)

18.336335510683693

In [ ]:
#print(ytestT[:,0]*(180/np.pi))
print(ytestT[:,0]*(180/np.pi))
a = ytestT[:,0]*(180/np.pi)
print(a.size())
np.savetxt("test.csv", a.numpy(), delimiter=",")

In [ ]:
#a = np.asarray([ [1,2,3], [4,5,6], [7,8,9] ])
#np.savetxt("foo.csv", a, delimiter=",")

In [ ]:
#print(finalpred.data[:,0]*(180/np.pi))
print(finalpred.data[:,0]*(180/np.pi))
b = finalpred.data[:,0]*(180/np.pi)
c=torch.abs(ytestT[:,0]*(180/np.pi)- finalpred.data[:,0]*(180/np.pi))
np.savetxt("pred.csv", b.numpy(), delimiter=",")

np.savetxt("diff.csv", c.numpy(), delimiter=",")

In [ ]:
finalpred.data[1,0]

In [ ]:
import scipy
from scipy.misc import imresize, imread, imshow
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['image.interpolation'] = 'none'
%matplotlib inline

fig = plt.figure()
fig.set_figheight(9)
fig.set_figwidth(9)
import cv2

test = cv2.imread("./Test_Net_image/4.JPG")
print(test.shape)
test = imresize(test,(180,320,3))
#imshow(test)
test = test.transpose(2,0,1)
test = np.reshape(test,(1,3,180,320))
test = test.astype(np.float32)
testPT = torch.from_numpy(test).float()
testPT = batch_rgb_to_bgr(testPT)
testPT = torch.div(testPT,255.0)
mn = [0.406,0.456,0.485]
sd = [0.225,0.224,0.229]
norm = Normalize(mn,sd)
testPT = norm(testPT)
'''
ind = 2000
testPT = xtestT[ind]
print("Actual angle==="+str(ytestT[ind,0]*(180/np.pi)), ytestT[ind,0])
testPT = testPT.view(1,3,180,320)
#'''
test_pred = net.forward(Variable(testPT, volatile=True).cuda())
print("Angle==="+str(test_pred.data[0,0]*(180/np.pi)), test_pred.data[0,0])
testx = testPT.numpy()
testx = np.reshape(testx,(3,180,320))
testx = testx.transpose(1,2,0)
testx = imresize(testx,(180,320,3))
#imshow(testx)
scipy.misc.imsave('test.png', testx)
a=fig.add_subplot(1,2,1)
imgplot = plt.imshow(testx)
a.set_title('Input')
a.axes.get_xaxis().set_visible(False)
a.axes.get_yaxis().set_visible(False)